Pandas Primer
==========

A collection of code snippets to serve as a reference point for performing pandas operations.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Read
--------

In [ ]:
df = pd.read_csv('ninja_pv_wind_profiles_singleindex.csv')

In [ ]:
def ReadFile(filename) :
    if 'csv' in filename:
        df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
    elif 'xls' in filename:
        df = pd.read_excel(io.BytesIO(decoded))
    elif 'json' in filename :
        df = pd.read_json(io.BytesIO(decoded))

    return df

Shape, Columns, summary, etc
------------

In [ ]:
df.shape

In [ ]:
df

In [ ]:
# Make the dataset smaller during setup
df = df[:1000]

In [ ]:
# A quick look at the dataset
df.describe()

Convert date columns to DateTime object
=========

In [ ]:
df['time_dt'] = pd.to_datetime(df['time'])

In [ ]:
type(df['time_dt'])

In [ ]:
for i in df.columns :
    if 'AT' in i : print(i)

In [ ]:
fig,ax = plt.subplots(figsize=[30,10])
ax.plot(np.array(df['time_dt'][:200]),np.array(df['AT_pv_national_current'][:200]),label='AT')
ax.plot(np.array(df['time_dt'][:200]),np.array(df['AT_wind_national_current'][:200]),label='AT wind')
ax.plot(np.array(df['time_dt'][:200]),np.array(df['BE_pv_national_current'][:200]),label='Belgium solar')
ax.plot(np.array(df['time_dt'][:200]),np.array(df['BE_wind_offshore_current'][:200]),label='Belgium wind offshore')
ax.plot(np.array(df['time_dt'][:200]),np.array(df['SE_pv_national_current'][:200]),label='Sweden')
ax.legend()
plt.show()

In [ ]:
fig2,ax2 = plt.subplots(figsize=[10,10])
ax2.scatter(df['AT_wind_national_current'],df['BE_wind_offshore_current'],label='AT/BE wind corr')
plt.show()

In [ ]:
# NOT WORKING
#df.resample('7D')

Column Manipulation
--------

**Shift**: e.g.
```python
df['deviceTime_end'] = df['deviceTime'].shift(1)
```

In [ ]:
def Meaningless() :
    # Subset that has the BG settings
    pd_smbg = pd_all[pd_all['type'] == 'smbg'][['deviceTime','value']]
    #pd_smbg['deviceTime_dt'] = pd.to_datetime(pd_all['deviceTime'])

    # Subset for containers
    columns_to_save = ['deviceTime','type','subType','duration','normal','extended','carbInput','value']
    pd_containers = pd_all[(pd_all['type'] == 'wizard') | (pd_all['type'] == 'bolus')][columns_to_save]

    # Subset for basal
    columns_to_save = ['deviceTime','deliveryType','percent','rate','suppressed','duration']

    # Step 1: Pick out all the basals
    pd_tmp1 = pd_all[(pd_all['type'] == 'basal')][columns_to_save]

    # Step 2: Save only temp, suspend, or entries just after temp, suspend
    # [:] to avoid a SettingWithCopyWarning
    pd_tmp2 = pd_tmp1[:][(pd_tmp1['deliveryType'] == 'temp') | (pd_tmp1['deliveryType'].shift(-1) == 'temp') |
                         (pd_tmp1['deliveryType'] == 'suspend') | (pd_tmp1['deliveryType'].shift(-1) == 'suspend')
                         ]

Concatenation (instead of `df['x'] = list(...)`)
-------

In [ ]:
a = pd.DataFrame({'a':list(range(10))})
b = pd.DataFrame({'b':list(range(10)),'c':list(range(10))})
pd.concat([pd.DataFrame(),a],axis=1)
pd.concat([a,b],axis=1)

Using np.Vectorize
--------

In [ ]:
def Meaningless() :
    def getDeviceTimeEndFixed(deviceTime,deviceTime_end,duration) :
        scheduled_end = pd.to_datetime(deviceTime) + datetime.timedelta(milliseconds=duration)
        if type(deviceTime_end) != type('') :
            return scheduled_end.strftime('%Y-%m-%dT%H:%M:%S')
        return min(pd.to_datetime(deviceTime_end),scheduled_end).strftime('%Y-%m-%dT%H:%M:%S')

    # Step 3: Save the end-times of temp and suspend based on this skimmed pd. Save fixed percent.

    pd_tmp2['deviceTime_end_fixed'] = np.vectorize(getDeviceTimeEndFixed)(pd_tmp2['deviceTime'],pd_tmp2['deviceTime_end'],pd_tmp2['duration'])


Handling missing or corrupted data
-----------
`isnull()`, `dropna()`, `fillna(0)` are examples

Datetime manipulation
-------

```
pd.to_datetime(bgs.iloc[i]['deviceTime']) < start_time_dt64 :
```

Merge (or join)
=======

In [ ]:
def fix() :
    pd.merge(a,b,how='outer',on='blah',suffixes=['asdf','sdfg'])
    
# There is also something called join..!